In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

# Encontrar todas as marcas
url = 'https://www.drogariasminasmais.com.br/medicamentos'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

marcas = soup.find_all("label", class_="vtex-checkbox__label w-100 c-on-base pointer")
marcas_corr = []
for marca in marcas:
    marcas_corr.append(str(marca).split('>')[1].split('<')[0])
for i in range(len(marcas_corr)):
    marcas_corr[i] = marcas_corr[i].lower().replace(" ", '-')
marcas_corr = marcas_corr[19:]

print(marcas_corr)

['hypera', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'cimed', 'germed-generico', 'libbs', 'apsen', 'biolab-rx', 'supera', 'medley-genericos', 'momenta', 'farmoquimica', 'eurofarma-generico', 'myralis', 'abbott-rx', 'teuto-generico', 'ache-otc', 'sanofi-rx', 'cristalia/latinofarma', 'sanofi-otc', 'prati-donaduzzi-generico', 'torrent', 'legrand-generico', 'biolab-generico', 'uniao-quimica-similar', 'neo-quimica-generico', 'uniao-quimica-genom', 'pharlab', 'neo-quimica-similar', 'nova-quimica-generico', 'geolab-generico', 'gsk-otc', 'marjan', 'novartis-rx', 'zanphy', 'gsk-rx', 'uniao-quimica-generico', 'procter--gambl', 'biolab-avert-otc', 'merck-rx', 'alcon', 'biosintetica-generico', 'servier', 'allergan', 'teuto-similar', 'ems-similar', 'bayer-rx', 'globo-generico', 'cellera', 'astrazeneca', 'sandoz-generico', 'herbarium', 'divcom', 'airela', 'zodiac-adium', 'sankyo-pharma', 'ranbaxy', 'natulab', 'bayer-otc', 'takeda', 'legrand-rx', 'boehringer', 'sigma-pharma-otc', 

In [25]:
def extrair_valores_por_chave(string, chave, categorico):
    valores = []  # Lista para armazenar os valores encontrados
    
    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')
    
    if categorico:
        # Enquanto houver ocorrências da chave
        while indice_chave != -1:
            # Encontrar o índice do próximo caractere após a ocorrência da chave
            indice_inicio_valor = indice_chave + len(f'"{chave}":') + 1
            
            # Encontrar o índice do próximo caractere que indica o final do valor
            indice_fim_valor = string.find('"', indice_inicio_valor)
            
            # Pegar o valor entre os índices de início e fim
            valor = string[indice_inicio_valor:indice_fim_valor]
            valores.append(valor)
            
            # Encontrar a próxima ocorrência da chave após a posição atual
            indice_chave = string.find(f'"{chave}":', indice_fim_valor)
        
    
    else:
        while indice_chave != -1:
                # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
                indice_inicio_valor = indice_chave + len(f'"{chave}":')
                
                # Encontrar o índice do próximo caractere que indica o final do preço
                indice_fim_valor = indice_inicio_valor
                while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
                    indice_fim_valor += 1
                
                # Pegar o valor entre os índices de início e fim
                valor = string[indice_inicio_valor:indice_fim_valor]
                valores.append(float(valor))
                
                # Encontrar a próxima ocorrência de '"Price":' após a posição atual
                indice_chave = string.find(f'"{chave}":', indice_fim_valor)
            
    return valores

# Exemplo de uso
texto = '... "productName":"Produto A", "productName":"Produto B", ...'
chave = "productName"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['Produto A', 'Produto B']

texto = '... "productReference":"123", "productReference":"45689", ...'
chave = "productReference"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['123', '45689']

texto = '... "Price":10.5, "Price":20.75, ...'
print(extrair_valores_por_chave(texto,"Price", False))  # Saída: [10.5, 20.75]

texto = '... "ean":"1234567890123", "ean":"9876543210987", ...'
print(extrair_valores_por_chave(texto,"ean",True))  # Saída: ['1234567890123', '9876543210987']


['Produto A', 'Produto B']
['123', '45689']
[10.5, 20.75]
['1234567890123', '9876543210987']


In [26]:



def extrair_quantidade(string):
    chave  = "recordsFiltered"

    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')

    # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
    indice_inicio_valor = indice_chave + len(f'"{chave}":')
    
    # Encontrar o índice do próximo caractere que indica o final do preço
    indice_fim_valor = indice_inicio_valor
    while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
        indice_fim_valor += 1
    
    # Pegar o valor entre os índices de início e fim
    quantidade = int(string[indice_inicio_valor:indice_fim_valor])
                    
    return quantidade


texto = '... "recordsFiltered":366, "ean":"9876543210987", ...'
print(extrair_quantidade(texto))  # Saída: ['1234567890123', '9876543210987']

366


In [27]:
colunas = ['Nome', 'Preco', 'Preco Sem Desconto', 'EAN', 'SKU', 'Marca']

# Criar DataFrame vazio com as colunas especificadas
df = pd.DataFrame(columns=colunas)

# Mostrar DataFrame vazio
print(df)

Empty DataFrame
Columns: [Nome, Preco, Preco Sem Desconto, EAN, SKU, Marca]
Index: []


In [28]:
for marca in marcas_corr:

    #primeira requisicao para conseguir quantidade de produtos por marcas

    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1'
    print(url)

    # fazendo a requisição com o site 
    requisicao = requests.get(url)

    quantidade = extrair_quantidade(requisicao.text)
    print(quantidade)

    #criação das listas dos dados
    ean_lista = []
    sku_lista = []
    nome_lista = []
    preco_lista = []
    precosemdesconto_lista = []

    pagina = 1

    while pagina <= math.ceil(quantidade/15):

        url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=' + str(pagina)

        # fazendo a requisição com o site 
        requisicao = requests.get(url)


        ean_lista.extend( extrair_valores_por_chave(requisicao.text,"ean",True))
        sku_lista.extend( extrair_valores_por_chave(requisicao.text, "productReference",True))
        nome_lista.extend( extrair_valores_por_chave(requisicao.text,"productName",True))
        preco_lista.extend( extrair_valores_por_chave(requisicao.text,'Price',False))
        precosemdesconto_lista.extend( extrair_valores_por_chave(requisicao.text,'ListPrice',False))
    # categoria_lista = extrair_valores_por_chave(requisicao.text,"originalName",True)
        
        pagina += 1

    print(nome_lista)

https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
366
['Coledue-R 10\\u002F20mg com 30 Cápsulas - Hypera', 'Aviv 100mg com 30 Comprimidos - Hypera', 'Ecoxe 90mg com 14 Comprimidos - Hypera', 'Ecoxe 90mg com 7 Comprimidos - Hypera', 'Aviv 50mg com 30 Comprimidos - Hypera', 'Ecoxe 60mg com 14 Comprimidos - Hypera', 'Coledue 10mg com 30 Cápsulas - Hypera', 'Coledue-R 10\\u002F10mg com 30 Cápsulas - Hypera', 'Alivium 50mg em Gotas com 30ml - Hypera', 'Celestone Injetável 4 mg com 1 Ampola - Hypera', 'Celestone Elixir 120 ml - Hypera', 'CORISTINA D CONGEST C\\u002F12 COMP', 'Eparema com 10 Drágeas - Hypera', 'Melatonum Max 30ml - Hypera', 'Addera Max com 30 Comprimidos - Hypera', 'Tamarine Fibras Max Laranja com 10 Sachês - Hypera', 'Picbam 5mg com 60 Comprimidos - Hypera', 'Addera+Imunidade Max com 30 Comprimidos - Hypera', 'Picbam 2,5mg com 60 Comprimidos - Hypera', 'Gastrol Abacaxi com 6 Envelopes - Hypera', 'Gast

o que adicionar:

botar as colunas no dataframe
a marca